In [15]:
%matplotlib inline

import pandas as pd
import numpy as np
import datetime
import time
from datetime import datetime, timedelta
import csv

# Define the path for both the source & output csv files

folder_directory = 'C:/Users/jones.cs/Documents/Dekunu/'

# Name of source file, output file is source file name prefixed by 'out'

file_name = 'dekunu_in.csv'
file_path = folder_directory+file_name
df = pd.read_csv(file_path, header=0, sep=',')

print(df)

      Timestamp  DeviceMode  gpsTimeValid     gpsTime  gpsTimeCentiSec  \
0       1241872           2             1  1523088338               50   
1       1242872           2             1  1523088339               50   
2       1243872           2             1  1523088340               30   
3       1244872           2             1  1523088341               30   
4       1245872           2             1  1523088342               30   
5       1246872           2             1  1523088343               40   
6       1247872           2             1  1523088344               30   
7       1248872           2             1  1523088345               30   
8       1249872           2             1  1523088346               20   
9       1250872           2             1  1523088347               30   
10      1251872           2             1  1523088348               40   
11      1252872           2             1  1523088349               20   
12      1253872           2           

In [43]:
# Combine the gpsTime with centiseconds and apply timezome shift, timezone needs doing properly!!!
tz = 2 * 3600
df['time'] = df['gpsTime'] + tz + df['gpsTimeCentiSec']/100

# time delta from epoch start @ 1970-01-01UTC + (elapsed time - (leap_count(2014) - leap_count(1980)))
df['time'] = datetime(1970, 1, 1) + pd.to_timedelta(df['time'] - (35 - 19), unit='s')

# Apply gSwoop time format e.g. 2016-09-10T14:20:51.40Z
df['time'] = df['time'].dt.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

df['velD'] = df['instVertSpeedMetersPerSec'] * -1 # Down is positive
df['groundspeed'] = df['gpsSpeedKnot'] * 0.514444 # Convert from knots to meters per second
df['lat'] = df['gpsLatitude']/1e6 # lat needs six deimal places
df['lon'] = df['gpsLongitude']/1e6 # Lon needs six deimal places

# Calculate North & East velocity components
conditions = [
    (df['gpsAngleDegree'] >= 0) & (df['gpsAngleDegree'] < 90),
    (df['gpsAngleDegree'] >= 90) & (df['gpsAngleDegree'] < 180),
    (df['gpsAngleDegree'] >= 180) & (df['gpsAngleDegree'] < 270),
    (df['gpsAngleDegree'] >= 270) & (df['gpsAngleDegree'] < 360)]

velN_calc = [
    (  df['groundspeed'] * np.cos(np.radians(df['gpsAngleDegree']))),
    (- df['groundspeed'] * np.cos(np.radians(180 - df['gpsAngleDegree']))),
    (- df['groundspeed'] * np.sin(np.radians(270 - df['gpsAngleDegree']))),
    (  df['groundspeed'] * np.cos(np.radians(360 - df['gpsAngleDegree'])))]

velE_calc = [
    (  df['groundspeed'] * np.sin(np.radians(df['gpsAngleDegree']))),
    (  df['groundspeed'] * np.sin(np.radians(180 - df['gpsAngleDegree']))),
    (- df['groundspeed'] * np.cos(np.radians(270 - df['gpsAngleDegree']))),
    (- df['groundspeed'] * np.sin(np.radians(360 - df['gpsAngleDegree'])))]

df['velN'] = np.select(conditions, velN_calc)
df['velE'] = np.select(conditions, velE_calc)

# Fix column naming and create dummy data for missing columns
df['heading'] = df['gpsAngleDegree']
df['numSV'] = df['gpsNumOfSats']
df['hMSL'] = df['altitudeAboveGroundMeters']
df['hAcc'] = 1
df['vAcc'] = 1
df['sAcc'] = 1
df['cAcc'] = 1
df['gpsFix'] = 1

headers1 = ['time','lat','lon','hMSL','velN','velE','velD','hAcc','vAcc','sAcc','heading','cAcc','gpsFix','numSV']
headers2 = ['','(deg)','(deg)','(m)','(m/s)','(m/s)','(m/s)','(m)','(m)','(m/s)','(deg)','(deg)','','']

out = df[headers1]
out = out.groupby('time').mean().reset_index()
out.loc[0] = headers2

print(out)


                             time      lat      lon   hMSL       velN  \
0                                    (deg)    (deg)    (m)      (m/s)   
1     2018-04-07T10:05:23.500000Z  51.0019  5.05978   54.7   -26.5153   
2     2018-04-07T10:05:24.300000Z  51.0017  5.05935     60   -29.2185   
3     2018-04-07T10:05:25.300000Z  51.0015  5.05884   63.9   -32.6656   
4     2018-04-07T10:05:26.300000Z  51.0011  5.05837   67.5   -36.3157   
5     2018-04-07T10:05:27.400000Z  51.0008   5.0579   73.2   -37.2776   
6     2018-04-07T10:05:28.300000Z  51.0005  5.05756   79.4   -40.1732   
7     2018-04-07T10:05:29.300000Z  51.0001  5.05725   81.7    -41.631   
8     2018-04-07T10:05:30.200000Z  50.9997  5.05701   87.5   -43.3151   
9     2018-04-07T10:05:31.300000Z  50.9993  5.05679   93.7    -45.874   
10    2018-04-07T10:05:32.400000Z  50.9988  5.05664   99.8   -45.1686   
11    2018-04-07T10:05:33.200000Z  50.9985  5.05658  104.8   -45.2813   
12    2018-04-07T10:05:34.300000Z  50.9981  5.05656

In [45]:
# Write results to csv, output file is source file name prefixed by 'out'

file_path_out = folder_directory+'out_'+file_name

out.to_csv(path_or_buf=file_path_out, 
          sep=',', 
          mode='a',
          header = True, # This keeps the header in the csv file
          index= False)  # Index False removes the row numbers present in the Datatable

